In [1]:
import pandas as pd
import numpy as np
import CRPS.CRPS as pscore
from KDEpy import FFTKDE
from KDEpy.bw_selection import silvermans_rule, improved_sheather_jones

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import random
random.seed(42)
np.random.seed(42)

In [3]:
actual1=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/section 4/actual.xlsx')
#hybrid models
#pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/trmg/trmg.xlsx')
#pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/trqg/trqg.xlsx')
#pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/wrgg/wrgg.xlsx')
#pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/vrlg/vrlg.xlsx')
#pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/ermg/ermg.xlsx')
#pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/tsmg/tsmg.xlsx')

#single models
#pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/qrnn2/qrnn2pred.xlsx')
#pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/mcqrnn/mcqrnn2pred.xlsx')
#pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/qrf/predqrf.xlsx')
#pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/qrlasso/lassopred.xlsx')

#5.5 ablation
#pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/5.4 Ablation analysis/tfmg/tfmg.xlsx')
#pred=pd.read_excel('/Users/ym_ya/Desktop/windpower/Simulation code and data/5.4 Ablation analysis/trq1g/trqg1.xlsx')

#5.4 sensitivity
pred=pd.read_excel('C:/Users/ym_ya/Desktop/windpower/Simulation code and data/5.4/5.4.xlsx')#first_trmg

In [4]:
actual1=np.array(actual1)
pred1=np.array(pred)

In [5]:
from math import sqrt
def calculate_metrics_with_ci(actual, pred1, n_iterations=1000, alpha=0.05):

    n = len(actual)
    metrics = {
        'RMSE': [],
        'MAE': [],
    }

    for _ in range(n_iterations):

        indices = np.random.choice(n, n, replace=True)
        actual_resampled = actual[indices]
        pred1_resampled = pred1[indices]

        rmse = sqrt(mean_squared_error(actual_resampled , pred1_resampled))
        metrics['RMSE'].append(rmse)

        mae = mean_absolute_error(actual_resampled , pred1_resampled)
        metrics['MAE'].append(mae)

    confidence_intervals = {}
    for metric, values in metrics.items():
        valid_values = [v for v in values if not np.isnan(v)]
        if valid_values:
            sorted_values = np.sort(valid_values)
            lower_idx = int(alpha/2 * len(sorted_values))
            upper_idx = int((1 - alpha/2) * len(sorted_values))
            confidence_intervals[metric] = (sorted_values[lower_idx], sorted_values[upper_idx])
        else:
            confidence_intervals[metric] = (np.nan, np.nan)

    point_estimates = {
        'RMSE': sqrt(mean_squared_error(actual, pred1)),
        'MAE': mean_absolute_error(actual, pred1),
    }

    return point_estimates, confidence_intervals

In [6]:
point_estimates, confidence_intervals = calculate_metrics_with_ci(actual1.reshape(-1), pred1[:,9].reshape(-1))
print("95% CI point prediction results:")
for metric in ['RMSE', 'MAE',]:
    mean_val = point_estimates[metric]
    lower, upper = confidence_intervals[metric]
    print(f"{metric}: {mean_val:.3f} [{lower:.3f}, {upper:.3f}]")

95% CI point prediction results:
RMSE: 120.474 [95.141, 159.348]
MAE: 62.141 [58.518, 66.449]


In [7]:
def calculate_interval_metrics_with_ci(actual, pred, n_iterations=1000, alpha=0.05):

    n = len(actual)
    quantiles = np.arange(0.05, 1.0, 0.05)
    metrics = {
        'Pinball Loss': [],
        'PICP90': [],
        'PINAW90': [],
    }
    pinball_per_sample = np.zeros(n)
    pl_ci = []
    for j, q in enumerate(quantiles):
        for i in range(n):
            if actual[i] < pred[i, j]:
                loss = (1 - q) * (pred[i, j] - actual[i])
            else:
                loss = q * (actual[i] - pred[i, j])
            pinball_per_sample[i] += loss

    pinball_per_sample /= len(quantiles)
    pl_mean = np.mean(pinball_per_sample)

    picp90_per_sample = np.zeros(n)
    pinaw90_per_sample = np.zeros(n)
    data_range = np.max(actual) - np.min(actual)

    for i in range(n):
        # PICP90 (1 if covered, 0 otherwise)
        picp90_per_sample[i] = 1 if (actual[i] >= pred[i, 0] and actual[i] <= pred[i, 18]) else 0

        # PINAW90
        pinaw90_per_sample[i] = (pred[i, 18] - pred[i, 0]) / data_range

    for _ in range(n_iterations):
        # 有放回抽样
        indices = np.random.choice(n, n, replace=True)

        # 计算所有指标
        metrics['Pinball Loss'].append(np.mean(pinball_per_sample[indices]))
        metrics['PICP90'].append(np.mean(picp90_per_sample[indices]))
        metrics['PINAW90'].append(np.mean(pinaw90_per_sample[indices]))
    # 计算置信区间
    confidence_intervals = {}
    for metric, values in metrics.items():
        sorted_values = np.sort(values)
        lower_idx = int(alpha/2 * len(sorted_values))
        upper_idx = int((1 - alpha/2) * len(sorted_values))
        confidence_intervals[metric] = (sorted_values[lower_idx], sorted_values[upper_idx])

    # 计算点估计值
    point_estimates = {
        'Pinball Loss': pl_mean,
        'PICP90': np.mean(picp90_per_sample),
        'PINAW90': np.mean(pinaw90_per_sample),
    }

    return point_estimates, confidence_intervals

In [8]:
point_estimates2, confidence_intervals2 = calculate_interval_metrics_with_ci(actual1,pred1)
print("95% Interval prediction results:")

for metric in point_estimates2:
    mean_val = point_estimates2[metric]
    lower, upper = confidence_intervals2[metric]
    print(f"{metric}: {mean_val:.3f} [{lower:.3f}, {upper:.3f}]")

C:\Users\ym_ya\AppData\Local\Temp\ipykernel_6944\488789964.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pinball_per_sample[i] += loss


95% Interval prediction results:
Pinball Loss: 26.445 [24.807, 28.530]
PICP90: 0.897 [0.886, 0.909]
PINAW90: 0.041 [0.041, 0.041]


In [9]:
def calculate_S(alpha, lower, upper, true):
    m = len(true)
    S = 0

    lower = np.asarray(lower).flatten()
    upper = np.asarray(upper).flatten()
    true = np.asarray(true).flatten()

    for i in range(m):
        di = upper[i] - lower[i]

        if true[i] < lower[i]:
            Si = -2 * alpha * di - 4 * (lower[i] - true[i])
        elif true[i] >= lower[i] and true[i] <= upper[i]:
            Si = -2 * alpha * di
        else:
            Si = -2 * alpha * di - 4 * (true[i] - upper[i])

        S += Si

    S /= m
    return S

Lower_sequence = pred1[:,0].flatten()
Upper_sequence = pred1[:,18].flatten()
actual1_flat = actual1.flatten()

alpha = 0.1
AIS = calculate_S(alpha, Lower_sequence, Upper_sequence,actual1_flat)


alpha1 = 0.05

bootstrap_scores = []
n = len(actual1)

# Bootstrap重采样
np.random.seed(42)
for _ in range(1000):

    indices = np.random.choice(n, n, replace=True)

    Li_sample = Lower_sequence[indices]
    Ui_sample = Upper_sequence[indices]
    true_sample = actual1[indices]

    score_b = calculate_S(alpha, Li_sample, Ui_sample, true_sample)
    bootstrap_scores.append(score_b)

lower_bound = np.percentile(bootstrap_scores, 2.5)
upper_bound = np.percentile(bootstrap_scores, 97.5)

print(f"Original SCORE: {AIS:.3f}")
print(f"95% Confidence Interval: [{lower_bound:.3f}, {upper_bound:.3f}]")

Original SCORE: -111.632
95% Confidence Interval: [-126.968, -99.310]


In [10]:
actual1n=np.squeeze(actual1)
print(actual1n.shape)

(2621,)


In [11]:
def calculate_crps_with_ci(actual, pred, n_bootstrap=1000, conf_level=0.95, sample_size=100):

    n = len(actual)
    crps_values = []
    random.seed(42)
    np.random.seed(42)

    std = [silvermans_rule(pred[i, :].reshape(-1, 1)) for i in range(n)]

    for i in range(n):

        resampled = np.random.choice(pred[i, :], size=sample_size, replace=True)
        resampled += np.random.randn(sample_size) * std[i]

        crps_val = pscore(resampled, actual[i]).compute()[0]
        crps_values.append(crps_val)

    crps_mean = np.mean(crps_values)
    np.random.seed(0)
    crps_boot = []

    for _ in range(n_bootstrap):

        indices = np.random.choice(n, n, replace=True)

        crps_boot.append(np.mean(np.array(crps_values)[indices]))

    crps_sorted = np.sort(crps_boot)
    alpha_ci = 1 - conf_level
    lower_idx = int(alpha_ci/2 * n_bootstrap)
    upper_idx = int((1 - alpha_ci/2) * n_bootstrap)

    return {
        "CRPS": crps_mean,
        "CI": (crps_sorted[lower_idx], crps_sorted[upper_idx]),
        "Bootstrap_values": crps_boot
    }

result = calculate_crps_with_ci(actual1n, pred1)
print(f"CRPS: {result['CRPS']:.3f}")
print(f"95% CI: [{result['CI'][0]:.3f}, {result['CI'][1]:.3f}]")

CRPS: 51.450
95% CI: [48.005, 55.482]


In [12]:
import gc
gc.collect()

0